<a href="https://colab.research.google.com/github/bopeng-sue/Optimal-Biofluid-Matrices-for-Human-Exposome-Biomonitoring/blob/main/preliminary_model_and_feature_screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import packages and file

In [ ]:
import numpy as np
import pandas as pd
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, MACCSkeys, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm
import warnings

warnings.filterwarnings('ignore')
RDLogger.DisableLog('rdApp.*')

In [ ]:
df = pd.read_excel('dataset_1211.xlsx')
df.head(2)

,Unnamed: 0,Biomonitor Compound PubChem CID,Blood study numbers,Urine study numbers,Tendency for the selection of blood samples,Biospecimen,CAS Number,Name,SMILES,hhlb(hours),...,JGI10,JGT,VE1_D,VE3_D,VR1_D,VR2_D,SRW5,AMW,WTPT-3,XLogP
0,0,70,0,2,0.0,Urine,816-66-0,4-Methyl-2-oxovaleric acid,CC(C)CC(=O)C(=O)O,0.61,...,0.0,0.601944,0.046505,-2.761367,35.180772,3.908975,0.0,6.845421,7.158910,0.904
1,1,89,0,1,0.0,Urine,484-78-6,Hydroxykynurenine,C1=CC(=C(C(=C1)O)N)C(=O)CC(C(=O)O)N,0.43,...,0.0,0.556317,0.130944,-3.252780,192.246721,12.015420,0.0,8.002847,14.772289,-2.464


# feature function

In [ ]:
def padel_descriptor(df: pd.DataFrame) -> np.ndarray:
    """
    Extracts Padel descriptors from a given DataFrame starting from column index 16
    to the end. It replaces NaN values with 0 and infinite values with 1e10.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing feature columns.

    Returns
    -------
    np.ndarray
        A NumPy array of transformed feature values.
    """
    # Slice columns from index 16 to the end
    X = df.iloc[:, 16:].values

    # Replace NaNs with 0
    X = np.nan_to_num(X, nan=0.0)

    # Replace infinities with a large finite value
    X[np.isinf(X)] = 1e10

    return X

In [ ]:
def maccs_descriptor(smiles_list):
    """
    Generates MACCS fingerprints for a list of SMILES strings and returns them as a DataFrame.

    Parameters
    ----------
    smiles_list : list of str
        A list of SMILES strings for which MACCS keys will be generated.

    Returns
    -------
    pd.DataFrame
        A DataFrame of MACCS bit values (0 or 1) with column names 'MACCS_1' through 'MACCS_167'.
    """
    # Generate MACCS fingerprints and convert them to a list of bits
    maccs_fingerprints = [
        list(MACCSkeys.GenMACCSKeys(Chem.MolFromSmiles(smiles)).ToBitString())
        for smiles in smiles_list
    ]

    # Convert the list of bits to a DataFrame
    # MACCSkeys are 167 bits long, hence 167 columns
    X = pd.DataFrame(maccs_fingerprints, columns=[f'MACCS_{i}' for i in range(1, 168)])

    # Convert strings to integers (0 and 1)
    X = X.astype(int)

    return X

In [ ]:
def ecfp_descriptor(smiles_list, radius=2, n_bits=1024):
    """
    Generates ECFP (Extended-Connectivity Fingerprints) for a list of SMILES strings
    and returns them as a NumPy array.

    Parameters
    ----------
    smiles_list : list of str
        A list of SMILES strings for which ECFP fingerprints will be generated.
    radius : int, optional (default=2)
        The fingerprint radius to use. For ECFP4, radius=2 is standard.
    n_bits : int, optional (default=1024)
        Length of the bit vector.

    Returns
    -------
    np.ndarray
        A 2D NumPy array where each row corresponds to the ECFP bit vector of the input SMILES.
    """
    def calculate_ecfp(smiles, radius=radius, n_bits=n_bits):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits, useFeatures=False)
            return np.array(fp)
        else:
            return np.zeros(n_bits, dtype=int)

    # Apply ECFP calculation to the input list of SMILES
    ecfp_fingerprints = np.array([calculate_ecfp(s) for s in smiles_list])
    return ecfp_fingerprints

In [ ]:
def fcfp_descriptor(smiles_list, radius=2, n_bits=1024):
    """
    Generates FCFP (Feature-based Circular Fingerprints) for a list of SMILES strings
    and returns them as a NumPy array.

    Parameters
    ----------
    smiles_list : list of str
        A list of SMILES strings for which FCFP fingerprints will be generated.
    radius : int, optional (default=2)
        The fingerprint radius to use.
    n_bits : int, optional (default=1024)
        Length of the bit vector.

    Returns
    -------
    np.ndarray
        A 2D NumPy array where each row corresponds to the FCFP bit vector of the input SMILES.
    """
    def calculate_fcfp(smiles, radius=radius, n_bits=n_bits):
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=radius, nBits=n_bits, useFeatures=True)
            return np.array(fp)
        else:
            return np.zeros(n_bits, dtype=int)

    # Apply FCFP calculation to the input list of SMILES
    fcfp_fingerprints = np.array([calculate_fcfp(s) for s in smiles_list])
    return fcfp_fingerprints

In [ ]:
def rdkit_descriptors_to_X(df: pd.DataFrame, smiles_col='SMILES', correlation_threshold=0.95) -> np.ndarray:
    """
    Compute RDKit molecular descriptors for each SMILES in the given DataFrame,
    remove descriptors with zero variance and highly correlated descriptors,
    and return the final feature matrix X.

    Parameters
    ----------
    df : pd.DataFrame
        A DataFrame containing at least a 'SMILES' column.
    smiles_col : str, optional
        The name of the column containing SMILES strings.
    correlation_threshold : float, optional
        Threshold above which descriptors are considered highly correlated and removed.

    Returns
    -------
    np.ndarray
        A 2D NumPy array containing the filtered RDKit descriptors.
    """
    # Convert SMILES to RDKit Mol objects
    mol_list = [Chem.MolFromSmiles(s) for s in df[smiles_col]]

    # Define the descriptor names
    descriptor_names = [desc_name[0] for desc_name in Descriptors._descList]

    # Create a descriptor calculator
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

    # Calculate descriptors for each molecule, if mol is None use NaNs
    descriptors_list = [
        calculator.CalcDescriptors(mol) if mol is not None else [np.nan]*len(descriptor_names)
        for mol in mol_list
    ]

    # Convert the list of descriptors to a DataFrame
    descriptors_df = pd.DataFrame(descriptors_list, columns=descriptor_names)

    # Replace NaNs with 0
    descriptors_df = descriptors_df.fillna(0)

    # Step 1: Remove descriptors with zero variance
    variance = descriptors_df.var()
    zero_variance_columns = variance[variance == 0].index
    descriptors_df = descriptors_df.drop(columns=zero_variance_columns)

    # Step 2: Remove descriptors with correlation exceeding the threshold
    corr = descriptors_df.corr().abs()
    to_drop = []

    for i, col1 in enumerate(corr.columns):
        for col2 in corr.columns[i+1:]:
            if corr.loc[col1, col2] > correlation_threshold:
                to_drop.append(col2)

    # Remove duplicates if any
    to_drop = list(set(to_drop))

    # Drop the highly correlated descriptors
    descriptors_df = descriptors_df.drop(columns=to_drop, errors='ignore')

    # Convert the final descriptors DataFrame to a NumPy array
    X = descriptors_df.values

    return X

# screening

In [ ]:
def run_descriptor_model_screening(df: pd.DataFrame,
                                   target_col: str = 'Tendency for the selection of blood samples',
                                   smiles_col: str = 'SMILES',
                                   descriptor_method: str = 'maccs',
                                   correlation_threshold: float = 0.95):
    """
    Apply the specified descriptor method to the input DataFrame, extract feature matrix X,
    then run multiple classification models with 5-fold stratified cross-validation and print the results.
    Includes a progress bar for the model evaluation loop.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing at least the target column and a SMILES column.
    target_col : str, optional
        Name of the target column.
    smiles_col : str, optional
        Name of the SMILES column.
    descriptor_method : str, optional
        The descriptor method to use. Choose from: 'padel', 'maccs', 'fcfp', 'ecfp', 'rdkit'.
    correlation_threshold : float, optional
        Used only by rdkit_descriptors_to_X if that method is chosen.

    Returns
    -------
    pd.DataFrame
        A DataFrame summarizing the results of each model with the chosen descriptor method.
    """

    # Extract and binarize the target if necessary
    y = (df[target_col] > 0.5).astype(int)

    # Compute X based on the chosen descriptor method
    if descriptor_method.lower() == 'padel':
        X = padel_descriptor(df)  # returns a numpy array
    elif descriptor_method.lower() == 'maccs':
        X = maccs_descriptor(df[smiles_col].tolist()).values
    elif descriptor_method.lower() == 'fcfp':
        X = fcfp_descriptor(df[smiles_col].tolist())  # returns numpy array
    elif descriptor_method.lower() == 'ecfp':
        X = ecfp_descriptor(df[smiles_col].tolist())  # returns numpy array
    elif descriptor_method.lower() == 'rdkit':
        X = rdkit_descriptors_to_X(df, smiles_col=smiles_col, correlation_threshold=correlation_threshold)
    else:
        raise ValueError("Invalid descriptor_method. Choose from 'padel', 'maccs', 'fcfp', 'ecfp', 'rdkit'.")

    # Define models
    models = {
        "Random Forest": RandomForestClassifier(random_state=42),
        "SVM": SVC(),
        "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
        "MLP": MLPClassifier(random_state=42, max_iter=1000),
        "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "Extra Trees": ExtraTreesClassifier(random_state=42),
        "LightGBM": LGBMClassifier(random_state=42, verbose=-1),
        "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
        "KNN": KNeighborsClassifier()
    }

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    results = {}

    # Use tqdm to show progress over models
    for name, model in tqdm(models.items(), desc=f"Evaluating models with {descriptor_method.upper()} descriptors"):
        accuracy_scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
        f1_scores = cross_val_score(model, X, y, cv=skf, scoring='f1_macro')
        precision_scores = cross_val_score(model, X, y, cv=skf, scoring='precision_macro')
        recall_scores = cross_val_score(model, X, y, cv=skf, scoring='recall_macro')

        results[name] = {
            "Accuracy (mean)": accuracy_scores.mean(),
            "Accuracy (std)": accuracy_scores.std(),
            "F1 (mean)": f1_scores.mean(),
            "F1 (std)": f1_scores.std(),
            "Precision (mean)": precision_scores.mean(),
            "Precision (std)": precision_scores.std(),
            "Recall (mean)": recall_scores.mean(),
            "Recall (std)": recall_scores.std()
        }
        print(f"{descriptor_method.upper()} - {name}: "
              f"Accuracy = {accuracy_scores.mean():.4f} ± {accuracy_scores.std():.4f}, "
              f"F1 = {f1_scores.mean():.4f} ± {f1_scores.std():.4f}, "
              f"Precision = {precision_scores.mean():.4f} ± {precision_scores.std():.4f}, "
              f"Recall = {recall_scores.mean():.4f} ± {recall_scores.std():.4f}")

    results_df = pd.DataFrame(results).T
    return results_df


In [ ]:
# Define the descriptor methods you want to test
descriptor_methods = ['padel', 'maccs', 'fcfp', 'ecfp', 'rdkit']

# Dictionary to store the results for each descriptor method
all_results = {}

for method in descriptor_methods:
    results = run_descriptor_model_screening(df,
                                             target_col='Tendency for the selection of blood samples',
                                             smiles_col='SMILES',
                                             descriptor_method=method)
    all_results[method] = results


for method, result_df in all_results.items():
    print(f"Results for {method.upper()}:")
    print(result_df)
    print("======================================")


Evaluating models with PADEL descriptors:   8%|▊         | 1/12 [00:14<02:35, 14.14s/it]

PADEL - Random Forest: Accuracy = 0.9104 ± 0.0152, F1 = 0.9069 ± 0.0150, Precision = 0.9037 ± 0.0161, Recall = 0.9136 ± 0.0113


Evaluating models with PADEL descriptors:  17%|█▋        | 2/12 [00:15<01:06,  6.69s/it]

PADEL - SVM: Accuracy = 0.6362 ± 0.0175, F1 = 0.4670 ± 0.0345, Precision = 0.6856 ± 0.1038, Recall = 0.5348 ± 0.0204


Evaluating models with PADEL descriptors:  25%|██▌       | 3/12 [00:52<03:05, 20.62s/it]

...

RDKIT - Extra Trees: Accuracy = 0.9254 ± 0.0176, F1 = 0.9223 ± 0.0180, Precision = 0.9191 ± 0.0180, Recall = 0.9286 ± 0.0169


Evaluating models with RDKIT descriptors:  83%|████████▎ | 10/12 [01:04<00:12,  6.23s/it]

RDKIT - LightGBM: Accuracy = 0.9236 ± 0.0258, F1 = 0.9202 ± 0.0272, Precision = 0.9168 ± 0.0262, Recall = 0.9262 ± 0.0295


Evaluating models with RDKIT descriptors: 100%|██████████| 12/12 [06:34<00:00, 32.92s/it]

RDKIT - CatBoost: Accuracy = 0.9217 ± 0.0150, F1 = 0.9188 ± 0.0153, Precision = 0.9144 ± 0.0152, Recall = 0.9274 ± 0.0145
RDKIT - KNN: Accuracy = 0.7574 ± 0.0198, F1 = 0.7402 ± 0.0187, Precision = 0.7455 ± 0.0214, Recall = 0.7374 ± 0.0169
Results for PADEL:
                     Accuracy (mean)  Accuracy (std)  F1 (mean)  F1 (std)  \
Random Forest               0.910436        0.015246   0.906857  0.015018   
SVM                         0.636154        0.017467   0.467033  0.034516   
XGBoost                     0.906767        0.015236   0.902542  0.016311   
MLP                         0.777985        0.019936   0.762673  0.022683   
Logistic Regression         0.828366        0.024015   0.823172  0.022805   
Decision Tree               0.850796        0.045203   0.841689  0.047874   
Gradient Boosting           0.893666        0.007331   0.888036  0.005965   
AdaBoost                    0.893700        0.030295   0.888624  0.030974   
Extra Trees                 0.925355        0.014

In [ ]:
combined_results = pd.concat(
    [df.assign(Descriptor_Method=method) for method, df in all_results.items()],
    ignore_index=True
)

# Save the combined results to a CSV file
combined_results.to_csv('all_descriptor_results.csv', index=False)

print("All results have been saved to 'all_descriptor_results.csv'.")

All results have been saved to 'all_descriptor_results.csv'.
